# Lattice computations

In [ ]:
import openmc
from IPython.display import Image

In this section, we will build one of the assemblies from the BEAVRS benchmark.  This is a PWR assembly with fuel pins, guide tubes, and borosilicate glass burnable poisons.  A diagram of th assembly is plotted below.  To make it a little easier (and improve our statistics!) we will only build one quarter of it.

In [ ]:
Image('assembly_diagram.png')

## Materials

Again we have UO2, Zr, and H2O.  We also have borosilicate glass (pyrex).  Note the shortcut for defining enriched uranium.  Also note that we can use `add_nuclide` and `add_element` directly with a string.  We do not have to create a `Nuclide` or `Element` object first.

In [ ]:
uo2 = openmc.Material(name='uo2')
uo2.add_element('U', 1.0, enrichment=3.0)
uo2.add_nuclide('O16', 2.0)
uo2.set_density('g/cm3', 10.0)

zirconium = openmc.Material(name='zirconium')
zirconium.add_element('Zr', 1.0)
zirconium.set_density('g/cm3', 6.55)

water = openmc.Material(name='watter')
water.add_nuclide('H1', 2)
water.add_nuclide('O16', 1)
water.set_density('g/cm3', 0.701)
water.add_s_alpha_beta('c_H_in_H2O')

pyrex = openmc.Material(name='pyrex')
pyrex.add_element('B', 0.49)
pyrex.add_element('O', 4.7)
pyrex.add_element('Al', 0.17)
pyrex.add_element('Si', 1.8)
pyrex.set_density('g/cm3', 2.26)

In [ ]:
mf = openmc.Materials((uo2, zirconium, water, pyrex))
mf.export_to_xml()

Later in this example, we will make a bunch of geometry plots.  By default, every region is colored randomly and the results are not always pretty.  Since we know our materials, let's define a dictonary of colors to use when plotting our geometry.

In [ ]:
colors = {}
colors[water] = (100, 200, 200)
colors[zirconium] = (150, 150, 150)
colors[pyrex] = (100, 255, 100)
#TODO: Define colors[uo2]

## Fuel pin

This is similar to the pincell example, but we don't have boundary conditions.  This `fuel_pin` universe extends to infinity

In [ ]:
pitch = 1.26

fuel_or = openmc.ZCylinder(R=0.39)
clad_ir = openmc.ZCylinder(R=0.40)
clad_or = openmc.ZCylinder(R=0.46)

fuel = openmc.Cell(1, 'fuel')
fuel.fill = uo2
fuel.region = -fuel_or

gap = openmc.Cell(2, 'air gap')
gap.fill = 'void'
gap.region = +fuel_or & -clad_ir

clad = openmc.Cell(3, 'clad')
clad.fill = zirconium
clad.region = +clad_ir & -clad_or

moderator = openmc.Cell(4, 'moderator')
moderator.fill = water
moderator.region = +clad_or

fuel_pin = openmc.Universe()
fuel_pin.add_cells((fuel, gap, clad, moderator))

When building a complex geometry, it is helpful to plot each universe as you go along.  Let's plot this pincell now

In [ ]:
# We need a cell to contain the fuel_pin universe.
main = openmc.Cell()
main.fill = fuel_pin

root = openmc.Universe()
root.add_cell(main)

g = openmc.Geometry()
g.root_universe = root
g.export_to_xml()

In [ ]:
p = openmc.Plot()
p.width = [pitch, pitch]
p.pixels = [400, 400]
p.color_by = 'material'
p.colors = colors

openmc.plot_inline(p)

## Guide tube

In [ ]:
clad_ir = openmc.ZCylinder(R=0.56)
clad_or = openmc.ZCylinder(R=0.60)

inner = openmc.Cell()
inner.fill = water
inner.region = -clad_ir

clad = openmc.Cell()
clad.fill = zirconium
clad.region = +clad_ir & -clad_or

outer = openmc.Cell()
outer.fill = water
outer.region = +clad_or

guide_tube = openmc.Universe()
guide_tube.add_cells((inner, clad, outer))

In [ ]:
main = openmc.Cell()
main.fill = guide_tube

root = openmc.Universe()
root.add_cell(main)

g = openmc.Geometry()
g.root_universe = root
g.export_to_xml()

In [ ]:
p = openmc.Plot()
p.width = [pitch, pitch]
p.pixels = [400, 400]
p.color_by = 'material'
p.colors = colors

openmc.plot_inline(p)

## Pyrex burnable poison

In [ ]:
Image("pyrex_diagram.png")

In [ ]:
# Define the cylinders which bound each radial zone.
radii = [0.21, 0.23, 0.24, 0.43, 0.44, 0.48, 0.56, 0.60]
cyls = [openmc.ZCylinder(R=R) for R in radii]

# Initialize a list of cells.
bp_cells = []

# Define the inner void zone first.
c = openmc.Cell()
c.region = -cyls[0]
c.fill = 'void'
bp_cells.append(c)

# Now all the sandwiched layers.
mats = [zirconium, 'void', pyrex, 'void', zirconium, water, zirconium]
for i in range(len(mats)):
    c = openmc.Cell()
    #TODO: Define the cell's region
    c.fill = mats[i]
    bp_cells.append(c)

# And the outer moderator region.
c = openmc.Cell()
c.region = +cyls[-1]
c.fill = water
bp_cells.append(c)

# Make a universe containing these cells
burn = openmc.Universe()
burn.add_cells(bp_cells)

In [ ]:
main = openmc.Cell()
main.fill = burn

root = openmc.Universe()
root.add_cell(main)

g = openmc.Geometry()
g.root_universe = root
g.export_to_xml()

In [ ]:
p = openmc.Plot()
p.width = [pitch, pitch]
p.pixels = [400, 400]
p.color_by = 'material'
p.colors = colors

openmc.plot_inline(p)

## Lattices in OpenMC

OpenMC has `RectLattice` and `HexLattice` objects.  This demo will use a `RectLattice`.  Let's look at a simple one before we do the quarter assembly.

First, we also need to define a universe that is all water.

In [ ]:
moderator = openmc.Cell()
moderator.fill = water

all_water = openmc.Universe()
all_water.add_cell(moderator)

In [ ]:
lattice = openmc.RectLattice()

# First we specify the dimension---the number of lattice tiles in x and y.
lattice.dimension = [2, 2]

# Next are the xy-coordiantes of the lower-left corner of the lattice.
lattice.lower_left = [0.0, 0.0]

# Next is the pitch---the size of the lattice tiles in each direction.
lattice.pitch = [pitch, pitch]

# Now we specify what is actually inside the lattice.  This 2D lattice will be a
# list of lists like
# lattice.universes = [ [a1, a2, ...], [b1, b2, ...], ...]
# The inner lists specify columns from left to right.  The outer lists specify
# rows from top to bottom.
lattice.universes = [
#TODO: Define a 2x2 lattice from fuel_pin, guide_tube, etc.
]

# We also need to specify what is outside of the lattice.  In this case, it is
# the infinite water universe.
lattice.outer = all_water

In [ ]:
main = openmc.Cell()
main.fill = lattice

root = openmc.Universe()
root.add_cell(main)

g = openmc.Geometry()
g.root_universe = root
g.export_to_xml()

In [ ]:
p = openmc.Plot()
p.origin = (pitch, pitch, 0)
p.width = (3*pitch, 3*pitch)
p.pixels = (400, 400)
p.color_by = 'material'
p.colors = colors

openmc.plot_inline(p)

## What exactly does `outer` mean?

We are taking that one universe and tiling it infinitely outside the lattice.

In [ ]:
# Try a different outer universe to see what I mean
lattice.outer = guide_tube

g.export_to_xml()

openmc.plot_inline(p)

## The BEAVRS assembly

In [ ]:
Image('assembly_diagram.png')

In [ ]:
lattice = openmc.RectLattice()

lattice.dimension = [9, 9]
lattice.pitch = [pitch, pitch]
lattice.outer = all_water

# I want (x0, y0) = (0, 0) to be the center of the instrument tube so that means
# the lower-left will be -half a pin pitch in x and y.
lattice.lower_left = [-pitch/2.0]*2

# Most of the lattice positions are fuel pins so rather than type all of those
# out, I will use a Python list comprehension to start with a 9x9 array of fuel.
lattice.universes = [[fuel_pin for i in range(9)] for j in range(9)]

# Then I will replace some fuel pins with guide tubes.  First index is the row,
# starting from the top, and the second is the column (like a matrix).
lattice.universes[2][0] = guide_tube
lattice.universes[2][3] = guide_tube
lattice.universes[5][0] = guide_tube
lattice.universes[5][3] = guide_tube
lattice.universes[5][6] = guide_tube
lattice.universes[8][0] = guide_tube
lattice.universes[8][3] = guide_tube
lattice.universes[8][6] = guide_tube

# And the burnable poison rod.
#TODO: Set one of the lattice universes to `burn` (the Pyrex burnable absorber)

Now we just have to add the boundary conditions and root universe to finish the geometry.

In [ ]:
height = 100 # Finite height is not strictly necessary but may avoid floating-point errors
assembly_pitch = 21.5
x0 = openmc.XPlane(x0=0.0, boundary_type='reflective')
x1 = openmc.XPlane(x0=assembly_pitch/2.0, boundary_type='reflective')
y0 = openmc.YPlane(y0=0.0, boundary_type='reflective')
y1 = openmc.YPlane(y0=assembly_pitch/2.0, boundary_type='reflective')
z0 = openmc.ZPlane(z0=-height/2.0, boundary_type='reflective')
z1 = openmc.ZPlane(z0=height/2.0, boundary_type='reflective')

main = openmc.Cell()
main.region = +x0 & -x1 & +y0 & -y1 & +z0 & -z1
main.fill = lattice

In [ ]:
root = openmc.Universe()
root.add_cell(main)

g = openmc.Geometry()
g.root_universe = root
g.export_to_xml()

In [ ]:
p = openmc.Plot()
p.origin = (0.25*assembly_pitch, 0.25*assembly_pitch, 0)
p.width = (0.55*assembly_pitch, 0.55*assembly_pitch)
p.pixels = (700, 700)
p.color_by = 'material'
p.colors = colors

openmc.plot_inline(p)

In [ ]:
settings = openmc.Settings()
settings.source = openmc.Source(space=openmc.stats.Box((0.1, 0.1, 0), (0.49*assembly_pitch, 0.49*assembly_pitch, 0)))
settings.batches = 50
settings.inactive = 10
settings.particles = 1000
settings.export_to_xml()

In [ ]:
openmc.run()

## Tallies

Okay, that was cool, but $k_\text{eff}$ isn't everyting.  We also want to know reaction rates so we can compute the power distribution, depletion rate, etc.  If we want pin-by-pin reaction rates, we have two options.  First, we can use a tally mesh.  This will lay a rectangular grid over the geometry and tally the reaction rates in each mesh bin.

In [ ]:
tallies = openmc.Tallies()

mesh = openmc.Mesh()
#mesh.dimension = lattice.dimension # Could set dimension from the lattice
mesh.dimension = [3, 3] # But we will use 3x3 for this example
mesh.lower_left = lattice.lower_left
mesh.width = lattice.pitch

mesh_filt = openmc.MeshFilter(mesh)

t = openmc.Tally(1)
t.filters = [mesh_filt]
t.scores = ['total', 'fission']
t.nuclides = ['total', 'U235']
tallies.append(t)

Or we can use something called a "distribcell" filter.  Note that the bin specifies a cell id.

In [ ]:
dist_filt = openmc.DistribcellFilter(fuel.id)
t = openmc.Tally(2)
t.filters = [dist_filt]
t.scores = ['total', 'fission']
t.nuclides = ['total', 'U235']
tallies.append(t)

tallies.export_to_xml()

In [ ]:
openmc.run()

In [ ]:
!cat tallies.out | head -n 15
print('...')
!cat tallies.out | tail -n 15

Notice the Distributed Cell description showing the path from the root universe all the way down to the fuel cell. This path can become a lot more complicated as we build more complex geometries: for instance, we could take our BEAVRS assembly and place it into a lattice as well.

In [ ]:
Image("core_diagram.png")

## Distributed materials

In a depletion problem, every fuel pin might need its own unique material.  We have a feature called "distributed materials" which makes this easier.

In [ ]:
many_uo2_mats = []
for i in range(72):
    m = openmc.Material(name='uo2')
    m.add_element('U', 1.0, enrichment=3.0)
    m.add_nuclide('O16', 2.0)
    m.set_density('g/cm3', 10.0)
    many_uo2_mats.append(m)

mf.add_materials(many_uo2_mats)
mf.export_to_xml()

In [ ]:
fuel.fill = many_uo2_mats
g.export_to_xml()
openmc.plot_inline(p)